# **Exploratory Data Analysis (EDA) approfondie**

---



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuration des graphiques
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

sns.set_palette("husl")

In [ ]:
# 1. Charger le fichier original
df = pd.read_csv('data/cleaned_paddydataset.csv', sep=',', encoding='utf-8', low_memory=False)

In [ ]:
# Aperçu des premières lignes
print("APERÇU DES DONNÉES:")
print("-" * 30)
print(df.head())
print()

In [ ]:
# Statistiques descriptives de base
print("STATISTIQUES DESCRIPTIVES:")
print("-" * 50)
print(df.describe())

In [ ]:
# Information sur la base
print(df.info())

In [ ]:
# Vérifier les valeurs null
print(df.isnull().sum().sum())

In [ ]:
# ----------------------------------------------------------------
# 3. DISTINCTION VARIABLES NUMÉRIQUES ET CATÉGORIQUES
# ----------------------------------------------------------------

print("\n" + "-"*70)
print("DISTINCTION VARIABLES NUMÉRIQUES ET CATÉGORIQUES")
print("-"*70)

# Identification des types de variables
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

print(f"Variables numériques: {len(numeric_features)}")
print(f"  Exemples: {numeric_features[:10]}")
print(f"\nVariables catégoriques: {len(categorical_features)}")
print(f"  Exemples: {categorical_features[:10]}")

In [ ]:
# Analyse rapide de la variable cible pendant l'EDA
target = 'Paddy yield(in Kg)'
"""
Rendement du riz en kilogrammes par parcelle.
C’est une variable continue → donc problème principal est un problème de régression.
"""
print(f"Variable cible : {target}")
print(f"Type : {df[target].dtype}")
print(f"Valeurs manquantes : {df[target].isnull().sum()}")
print("\nStatistiques descriptives :")
print(df[target].describe())

# Distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df[target], kde=True, color='skyblue')
plt.title(f'Distribution de {target}')
plt.xlabel(target)

plt.subplot(1, 2, 2)
sns.boxplot(y=df[target], color='lightgreen')
plt.title(f'Boxplot de {target}')

plt.tight_layout()
plt.show()

In [ ]:
# Visualisation de la distribution de paddy
fig, axes = plt.subplots(1, 2, figsize=(15, 12))

# Histogramme
axes[0].hist(df[target], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].axvline(df[target].mean(), color='red', linestyle='--', label=f'Moyenne: {df[target].mean():,.0f} Kg')
axes[0].axvline(df[target].median(), color='green', linestyle='--', label=f'Médiane: {df[target].median():,.0f} Kg')
axes[0].set_xlabel('Kg')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution de riz')
axes[0].legend()


# Q-Q plot pour tester la normalité
stats.probplot(df[target], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot - Test de normalité')

plt.tight_layout()
plt.show()

In [ ]:
target = 'Paddy yield(in Kg)'

# ========================================
# 1. Sélection des 12 variables les plus corrélées avec la cible
# ========================================
# Corrélation de toutes les variables numériques avec la cible
target_corr = df[numeric_features].corr()[target].drop(target)

# Sélection des 12 variables les plus corrélées (valeur absolue)
top_12_features = target_corr.abs().sort_values(ascending=False).head(12).index.tolist()

# Ajouter la cible à la liste
selected_features = [target] + top_12_features

print("\nTop 12 variables sélectionnées pour la corrélation :")
print(selected_features)

# ========================================
# 2. Matrice de corrélation pour les 12 variables
# ========================================
corr_matrix = df[selected_features].corr()

plt.figure(figsize=(14, 12))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},
    mask=np.abs(corr_matrix) < 0.01  # Masque les très faibles corrélations
)
plt.title('Matrice de Corrélation – Top 12 Variables Numériques',
          fontsize=18, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# ========================================
# 3. Corrélation avec la variable cible
# ========================================
print("\n" + "="*80)
print("CORRÉLATION AVEC LA VARIABLE CIBLE : Paddy yield(in Kg)")
print("="*80)

# Corrélations triées par valeur absolue
target_corr_selected = corr_matrix[target].drop(target)
target_corr_abs = target_corr_selected.abs().sort_values(ascending=False)

# Affichage des top 12 corrélations
print(pd.DataFrame({
    'Variable': target_corr_abs.index,
    'Corrélation': target_corr_selected[target_corr_abs.index].round(3)
}).to_string(index=False))


In [ ]:
# ========================================
# 4. Barplot des top corrélations
# ========================================
plt.figure(figsize=(10, 8))
sns.barplot(
    x=target_corr_abs.values,
    y=target_corr_abs.index,
    palette='viridis'
)
plt.title('Top 12 Variables les plus Corrélées avec Paddy yield',
          fontsize=14, fontweight='bold')
plt.xlabel('Corrélation Absolue')
plt.tight_layout()
plt.show()

In [ ]:
# ================================================================
# 6. EXPLORATION DES VARIABLES CATÉGORIQUES
# ================================================================

print("\n" + "="*60)
print("EXPLORATION DES VARIABLES CATÉGORIQUES")
print("="*60)

# Analyse des variables catégoriques
categorical_analysis = pd.DataFrame({
    'Variable': categorical_features,
    'Valeurs_Uniques': [df[col].nunique() for col in categorical_features],
    'Valeurs_Manquantes': [df[col].isnull().sum() for col in categorical_features],
    'Mode': [df[col].mode().iloc[0] if not df[col].mode().empty else 'N/A' for col in categorical_features],
    'Fréq_Mode': [df[col].value_counts().iloc[0] if len(df[col].value_counts()) > 0 else 0 for col in categorical_features]
})

print("ANALYSE DES VARIABLES CATÉGORIQUES:")
print("-" * 38)
print(categorical_analysis.head(10))

# ================================================================
# Impact des variables catégoriques sur le rendement en paddy
# ================================================================
print("\nIMPACT DES VARIABLES CATÉGORIQUES SUR LE PADDY YIELD:")
print("-" * 60)

categorical_impact = {}
for col in categorical_features:
    if df[col].notna().sum() > 0:  # Seulement si la variable a des valeurs non-null
        grouped = df.groupby(col)[target].agg(['mean', 'count']).reset_index()
        grouped = grouped[grouped['count'] >= 5]  # Au moins 5 observations
        if len(grouped) > 1:
            yield_variance = grouped['mean'].var()
            categorical_impact[col] = yield_variance

impact_sorted = sorted(categorical_impact.items(), key=lambda x: x[1], reverse=True)

print("Variables avec le plus d'impact sur le Paddy Yield (variance des moyennes):")
for var, impact in impact_sorted[:10]:
    print(f"{var}: {impact:,.2f} Kg^2")

# ================================================================
# Visualisation de l'impact des principales variables catégoriques
# ================================================================
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
axes = axes.ravel()

top_categorical = [var for var, _ in impact_sorted[:4]]  # Top 4 variables

for i, var in enumerate(top_categorical):
    if var in df.columns:
        # Calculer les moyennes par catégorie
        grouped = df.groupby(var)[target].agg(['mean', 'count']).reset_index()
        grouped = grouped[grouped['count'] >= 5].sort_values('mean', ascending=True)

        bars = axes[i].bar(range(len(grouped)), grouped['mean'], color='skyblue', edgecolor='black')
        axes[i].set_xticks(range(len(grouped)))
        axes[i].set_xticklabels(grouped[var], rotation=45, ha='right')
        axes[i].set_ylabel('Rendement moyen (Kg)')
        axes[i].set_title(f'Rendement moyen de Paddy par {var}')

        # Ajouter les valeurs sur les barres
        for j, bar in enumerate(bars):
            height = bar.get_height()
            axes[i].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                         f'{height:,.0f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# ================================================================
# 7. ANALYSE SPÉCIFIQUE DES BLOCS AGRICOLES (AGRIBLOCK)
# ================================================================

def categorical_target_analysis(df, categorical_col, target_col, top_n=10):
    """
    Analyse une variable catégorielle par rapport à une variable cible continue.

    Affiche :
    - Tableau des statistiques (moyenne, médiane, écart-type, nombre)
    - Barplot du top_n catégories par moyenne
    - Scatter plot du nombre d'échantillons vs moyenne
    - Annoter les top catégories

    Parameters:
    -----------
    df : pd.DataFrame
        Dataframe contenant les colonnes.
    categorical_col : str
        Nom de la variable catégorielle.
    target_col : str
        Nom de la variable cible continue.
    top_n : int
        Nombre de catégories à afficher dans le tableau et les annotations.
    """
    import matplotlib.pyplot as plt
    import seaborn as sns

    print("\n" + "="*60)
    print(f"ANALYSE DE '{categorical_col}' PAR RAPPORT À '{target_col}'")
    print("="*60)

    # Calcul des statistiques
    stats_df = df.groupby(categorical_col).agg({
        target_col: ['mean', 'median', 'count', 'std']
    }).round(2)
    stats_df.columns = ['Moyenne', 'Médiane', 'Nb', 'Écart_Type']
    stats_df = stats_df.sort_values('Moyenne', ascending=False)

    print(f"STATISTIQUES PAR {categorical_col.upper()} (Top {top_n}):")
    print("-"*35)
    display(stats_df.head(top_n))

    # Visualisations
    plt.figure(figsize=(15, 10))

    # Barplot du rendement moyen
    plt.subplot(2, 1, 1)
    plt.bar(range(len(stats_df)), stats_df['Moyenne'], color='lightgreen', edgecolor='black')
    plt.xticks(range(len(stats_df)), stats_df.index, rotation=45, ha='right')
    plt.ylabel(f'Moyenne de {target_col} (Kg)')
    plt.title(f'Moyenne de {target_col} par {categorical_col}')
    plt.grid(axis='y')

    # Scatter plot : nombre d'échantillons vs moyenne
    plt.subplot(2, 1, 2)
    plt.scatter(stats_df['Nb'], stats_df['Moyenne'], s=100, alpha=0.7, color='orange')
    plt.xlabel('Nombre d\'observations')
    plt.ylabel(f'Moyenne de {target_col} (Kg)')
    plt.title(f'Relation entre nombre d\'observations et moyenne de {target_col} par {categorical_col}')

    # Annoter les top_n catégories
    for i, cat in enumerate(stats_df.index[:top_n]):
        plt.annotate(cat,
                     (stats_df.loc[cat, 'Nb'], stats_df.loc[cat, 'Moyenne']),
                     xytext=(5, 5), textcoords='offset points', fontsize=8)

    plt.grid()
    plt.tight_layout()
    plt.show()

    return stats_df

In [ ]:
# Analyse par Agriblock
block_stats = categorical_target_analysis(df, 'Agriblock', 'Paddy yield(in Kg)')

# Analyse par Variety
variety_stats = categorical_target_analysis(df, 'Variety', 'Paddy yield(in Kg)')

# Analyse par Soil Types
soil_stats = categorical_target_analysis(df, 'Soil Types', 'Paddy yield(in Kg)')


In [ ]:
# ================================================================
# 8. ANALYSE TEMPORELLE AGRONOMIQUE
# ================================================================

print("\n" + "="*60)
print("ANALYSE TEMPORELLE AGRONOMIQUE")
print("="*60)

# Regrouper par périodes d'application d'intrants (en jours)
temporal_cols = ['DAP_20days', 'Urea_40Days', 'Potassh_50Days', 'Micronutrients_70Days', 'Pest_60Day(in ml)']

# Statistiques sur le rendement pour chaque période
temporal_stats = df[temporal_cols + ['Paddy yield(in Kg)']].agg(['mean', 'median', 'count']).round(2)
print("STATISTIQUES TEMPORELLES DES INTRANTS ET DU RENDEMENT:")
display(temporal_stats)

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(temporal_cols[:4]):  # On prend 4 intrants principaux pour le plot
    axes[i].plot(df[col], df['Paddy yield(in Kg)'], 'o', alpha=0.6)
    axes[i].set_xlabel(f'{col} (kg ou ml)')
    axes[i].set_ylabel('Rendement (Kg)')
    axes[i].set_title(f'Rendement vs {col}')
    axes[i].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# ================================================================
# 9. ANALYSE DE LA QUALITÉ AGRONOMIQUE ET DES TRAITEMENTS (PADDY)
# ================================================================

print("\n" + "="*60)
print("ANALYSE DE LA QUALITÉ AGRONOMIQUE ET DES TRAITEMENTS")
print("="*60)

# Variables représentatives de la qualité
quality_vars = ['Soil Types', 'Nursery', 'Variety', 'Pest_60Day(in ml)', 'Trash(in bundles)']

n_vars = len(quality_vars)
ncols = 2
nrows = int(np.ceil(n_vars / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*8, nrows*5))
axes = axes.ravel()  # Pour avoir un tableau plat

for i, var in enumerate(quality_vars):
    if var in df.columns:
        grouped = df.groupby(var)['Paddy yield(in Kg)'].mean()

        bars = axes[i].bar(range(len(grouped)), grouped.values, color='lightgreen')
        axes[i].set_xticks(range(len(grouped)))
        axes[i].set_xticklabels(grouped.index, rotation=45, ha='right')
        axes[i].set_ylabel('Rendement moyen (Kg)')
        axes[i].set_title(f'Rendement moyen par {var}')

        for j, bar in enumerate(bars):
            height = bar.get_height()
            axes[i].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                         f'{height:.1f}', ha='center', va='bottom', fontsize=9)

# Supprimer les sous-graphes inutilisés
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
sns.scatterplot(x='Hectares ', y=target, data=df)
sns.scatterplot(x='Nursery area (Cents)', y=target, data=df)
plt.title("Rendement vs Surface du champ / pépinière")
plt.show()


In [ ]:
# ================================================================
# 10. ANALYSE DES SURFACES ET DIMENSIONS (PADDY)
# ================================================================

print("\n" + "="*60)
print("ANALYSE DES SURFACES AGRONOMIQUES ET DIMENSIONS")
print("="*60)

# Variables de surface importantes
surface_vars = ['Hectares ', 'Nursery area (Cents)', 'LP_Mainfield(in Tonnes)', 'LP_nurseryarea(in Tonnes)',
                'Trash(in bundles)']
# Corrélations avec le rendement
surface_corr = df[surface_vars + ['Paddy yield(in Kg)']].corr()['Paddy yield(in Kg)'].sort_values(ascending=False)
print("CORRÉLATIONS DES SURFACES AVEC LE RENDEMENT:")
print("-" * 50)
print(surface_corr)

# Visualisation des relations surface-rendement
n_vars = len(surface_vars)
ncols = 2
nrows = int(np.ceil(n_vars / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*8, nrows*5))
axes = axes.ravel()

for i, var in enumerate(surface_vars):
    axes[i].scatter(df[var], df['Paddy yield(in Kg)'], alpha=0.5, color='green')
    axes[i].set_xlabel(f'{var}')
    axes[i].set_ylabel('Rendement (Kg)')
    axes[i].set_title(f'{var} vs Rendement\nCorr: {surface_corr[var]:.3f}')

    # Ligne de régression
    valid_data = df[[var, 'Paddy yield(in Kg)']].dropna()
    if len(valid_data) > 1:
        z = np.polyfit(valid_data[var], valid_data['Paddy yield(in Kg)'], 1)
        p = np.poly1d(z)
        axes[i].plot(valid_data[var], p(valid_data[var]), "r--", alpha=0.8)

# Supprimer les axes vides si nécessaire
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
weather_vars = [
    '30DRain( in mm)', '30_50DRain( in mm)',
    '51_70DRain(in mm)', '71_105DRain(in mm)',
    'Min temp_D1_D30', 'Max temp_D1_D30'
]

# Nombre de variables météo
n_vars = len(weather_vars)

plt.figure(figsize=(15, 4 * n_vars))

for i, var in enumerate(weather_vars):
    plt.subplot(n_vars, 1, i + 1)
    sns.scatterplot(x=df[var], y=df[target], alpha=0.6)
    plt.xlabel(var)
    plt.ylabel('Rendement (Kg)')
    plt.title(f'Rendement vs {var}')
    plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
pivot = df.pivot_table(values=target, index='Variety', columns='Soil Types', aggfunc='mean')
sns.heatmap(pivot, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title("Rendement moyen par Variety et Soil Types")
plt.show()



In [ ]:
# Distribution du rendement par variété
# Boxplots pour comparer la dispersion et détecter les variabilités entre variétés.
plt.figure(figsize=(10,6))
sns.boxplot(x='Variety', y='Paddy yield(in Kg)', data=df, palette="Set2")
plt.title("Rendement par variété de riz")
plt.ylabel("Rendement (Kg)")
plt.show()


In [ ]:
# Moyenne et écart-type par variété
# Table récapitulative pour voir quelles variétés sont les plus performantes et stables.
variety_stats = df.groupby('Variety')['Paddy yield(in Kg)'].agg(['mean','std','count']).sort_values('mean', ascending=False)
print(variety_stats)

In [ ]:
# Rendement vs fertilisation par variété
# Scatter plots ou barplots pour vérifier si certaines variétés réagissent mieux à certains fertilisants.
fertilizer_cols = ['DAP_20days','Urea_40Days','Potassh_50Days']
for fert in fertilizer_cols:
    plt.figure(figsize=(8,5))
    sns.scatterplot(x=fert, y='Paddy yield(in Kg)', hue='Variety', data=df)
    plt.title(f"Rendement vs {fert} par variété")
    plt.show()


In [ ]:
# Rendement vs surface par variété
# Vérifier si l’effet de la superficie du champ ou de la pépinière diffère selon la variété.
plt.figure(figsize=(10,6))
sns.scatterplot(x='Hectares ', y='Paddy yield(in Kg)', hue='Variety', data=df)
plt.title("Rendement vs superficie par variété")
plt.show()

In [ ]:
# Heatmap corrélations spécifiques à chaque variété
# Calculer les corrélations entre fertilisants/météo et rendement par variété pour détecter des patterns spécifiques.
for var in df['Variety'].unique():
    subset = df[df['Variety'] == var]
    corr = subset[['Paddy yield(in Kg)', 'DAP_20days','Urea_40Days','Potassh_50Days']].corr()
    plt.figure(figsize=(5,4))
    sns.heatmap(corr, annot=True, cmap="coolwarm", center=0)
    plt.title(f"Corrélations - {var}")
    plt.show()

In [ ]:
# Analyse de variance (ANOVA) par variété
# Tester statistiquement si les moyennes de rendement diffèrent entre variétés.
from scipy.stats import f_oneway

groups = [df[df['Variety']==v]['Paddy yield(in Kg)'].dropna() for v in df['Variety'].unique()]
f_stat, p_val = f_oneway(*groups)
print(f"ANOVA Variety: F={f_stat:.2f}, p={p_val:.4f}")
# p > 0.05, les rendements moyens ne diffèrent pas significativement entre variétés.

In [ ]:
# Distribution de la variable cible : Variety
target_counts = df['Variety'].value_counts()
target_percentage = df['Variety'].value_counts(normalize=True) * 100

print("Distribution de la variable cible 'Variety':")
print("=" * 60)
for variety in target_counts.index:
    print(f"{variety:15} : {target_counts[variety]:4d} échantillons ({target_percentage[variety]:.2f}%)")

# Tri des variétés par ordre alphabétique ou par count pour une meilleure lisibilité
target_counts = target_counts.sort_values(ascending=False)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 1. Bar plot
bars = axes[0].bar(target_counts.index, target_counts.values,
                   color=['#3498db', '#e67e22', '#27ae60'],
                   alpha=0.85, edgecolor='black', linewidth=1.2)
axes[0].set_xlabel('Variété de Paddy', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Nombre d\'échantillons', fontsize=12, fontweight='bold')
axes[0].set_title('Distribution de la variable cible (Variety)', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Ajouter les valeurs au-dessus des barres
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 5,
                 f'{int(height)}',
                 ha='center', va='bottom', fontsize=11, fontweight='bold')

# 2. Pie chart
colors = ['#3498db', '#e67e22', '#27ae60']
wedges, texts, autotexts = axes[1].pie(target_counts.values,
                                      labels=target_counts.index,
                                      autopct='%1.1f%%',
                                      colors=colors,
                                      startangle=90,
                                      wedgeprops={'edgecolor': 'black', 'linewidth': 1.5})

# Améliorer la lisibilité des pourcentages
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(11)

axes[1].set_title('Répartition des Variétés de Paddy', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()